# Lane Tracking with Kalman Filter
This notebook implements a Kalman Filter to track lane markings over time.
Tracking helps in:
1. Smoothing lane detections.
2. Predicting lane positions when detection fails.
3. Reducing jitter.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 1. Kalman Filter Implementation
We define a generic `KalmanFilter` class.
For lane tracking, we can model the state as:
$$ x = [position, velocity]^T $$
The measurement is just the position:
$$ z = [position] $$


In [ ]:
class KalmanFilter:
    def __init__(self, dt, u_x, u_z, std_acc, x_std_meas):
        """
        :param dt: sampling time (1/fps)
        :param u_x: acceleration magnitude
        :param u_z: measurement noise
        :param std_acc: process noise magnitude
        :param x_std_meas: standard deviation of the measurement
        """
        self.dt = dt
        self.u_x = u_x
        self.u_z = u_z
        self.std_acc = std_acc
        self.x_std_meas = x_std_meas
        
        # State vector [position, velocity]
        self.x = np.matrix([[0], [0]])
        
        # State Transition Matrix A
        # x_k+1 = x_k + v_k * dt
        # v_k+1 = v_k
        self.A = np.matrix([[1, self.dt],
                            [0, 1]])
        
        # Control Input Matrix B (assuming constant acceleration model, but usually 0 for tracking)
        self.B = np.matrix([[(self.dt**2)/2], [self.dt]]) 
        
        # Measurement Mapping Matrix H
        # We only measure position
        self.H = np.matrix([[1, 0]])
        
        # Process Noise Covariance Q
        self.Q = np.matrix([[(self.dt**4)/4, (self.dt**3)/2],
                            [(self.dt**3)/2, self.dt**2]]) * self.std_acc**2
        
        # Measurement Noise Covariance R
        self.R = np.matrix([[self.x_std_meas**2]])
        
        # Error Covariance Matrix P
        self.P = np.eye(self.A.shape[1])
        
    def predict(self):
        # Predicted state estimate
        # x_k|k-1 = A * x_k-1|k-1 + B * u
        self.x = np.dot(self.A, self.x) + np.dot(self.B, self.u_x)
        
        # Predicted error covariance
        # P_k|k-1 = A * P_k-1|k-1 * A^T + Q
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        return self.x
    
    def update(self, z):
        # Innovation (Measurement Residual)
        # y = z - H * x_k|k-1
        y = z - np.dot(self.H, self.x)
        
        # Innovation Covariance
        # S = H * P * H^T + R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        
        # Kalman Gain
        # K = P * H^T * S^-1
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        
        # Updated state estimate
        # x_k|k = x_k|k-1 + K * y
        self.x = self.x + np.dot(K, y)
        
        # Updated error covariance
        # P_k|k = (I - K * H) * P_k|k-1
        I = np.eye(self.H.shape[1])
        self.P = np.dot((I - np.dot(K, self.H)), self.P)
        return self.x


## 2. Simulation
Let's simulate a lane moving (e.g., changing curvature or lateral position) and see how the Kalman Filter tracks it.


In [ ]:
# Simulation parameters
dt = 0.1
t = np.arange(0, 10, dt)

# True state (sinusoidal motion to simulate lane curvature change)
true_position = np.sin(t)
true_velocity = np.cos(t) # derivative of sin(t)

# Measurements (True state + Noise)
noise_std = 0.1
measurements = true_position + np.random.normal(0, noise_std, len(t))

# Initialize Kalman Filter
kf = KalmanFilter(dt=dt, u_x=0, u_z=0, std_acc=1.0, x_std_meas=noise_std)

# Tracking
predictions = []
for z in measurements:
    kf.predict()
    x_hat = kf.update(np.array([[z]]))
    predictions.append(x_hat[0, 0])

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(t, true_position, label='True Position', linestyle='--')
plt.scatter(t, measurements, label='Measurements (Noisy)', color='r', s=10)
plt.plot(t, predictions, label='Kalman Filter Estimate', color='g')
plt.title('Lane Position Tracking with Kalman Filter')
plt.xlabel('Time (s)')
plt.ylabel('Position')
plt.legend()
plt.grid()
plt.show()


## 3. Application to Lane Detection
In the full pipeline, you would:
1. Detect lanes in each frame using the U-Net.
2. Extract lane points or coefficients (e.g., polynomial fit).
3. For each lane instance, update its corresponding Kalman Filter with the new measurement.
4. If a lane is not detected in a frame, use `kf.predict()` to estimate its position.
